# 2016 Presidential election16 Results

### Load python tools

In [669]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from earthpy import clip as cl
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

### Read and clean up dataframe

In [670]:
election16 = pd.read_csv('input/2016_US_County_Level_Presidential_Results.csv', \
dtype={'votes_dem': (int),'votes_gop':(int), 'total_votes': (int), 'per_dem': (str), \
        'per_gop': (str), 'diff': (str), 'per_point_diff': (str), 'combined_fips': (str)})
election16['diff'] = election16['diff'].str.replace(',','').astype(int)
election16['per_dem'] = ((election16['votes_dem']/election16['total_votes'])*100).round(2).astype(float)
election16['per_gop'] = ((election16['votes_gop']/election16['total_votes'])*100).round(2).astype(float)
election16['per_point_diff'] = (election16['per_point_diff'].str.replace('%','').str.replace('.','').astype(int) / 100)
election16 = election16.drop(columns=['Unnamed: 0'])
election16['fips'] = election16['combined_fips']
election16 = election16.drop(columns=['combined_fips'])
election16['fips'] = election16['fips'].str.zfill(5)

### Who won each place? 

In [703]:
election16['winner'] = (election16[['per_dem','per_gop']].idxmax(axis = 1)).str.replace('per_','')

In [704]:
election16.head(10)

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,fips,winner
0,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02013,gop
1,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02016,gop
2,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02020,gop
3,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02050,gop
4,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02060,gop
5,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02068,gop
6,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02070,gop
7,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02090,gop
8,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02100,gop
9,93003,130413,246588,37.72,52.89,37410,15.17,AK,Alaska,02105,gop


### Merge with county geography

In [712]:
counties = gpd.read_file('input/us_counties.geojson')
counties = counties.rename(str.lower, axis='columns')

In [713]:
results = gpd.GeoDataFrame(election16.merge(counties, on='fips'))

In [714]:
results.crs = {'init': "epsg:4326"}

In [715]:
results['area'] = results['geometry'].to_crs({'init': 'epsg:3310'})\
               .map(lambda p: p.area / 2.59e+6).round(2)

In [716]:
results.columns

Index(['votes_dem', 'votes_gop', 'total_votes', 'per_dem', 'per_gop', 'diff',
       'per_point_diff', 'state_abbr', 'county_name', 'fips', 'winner', 'name',
       'state_name', 'state_fips', 'cnty_fips', 'pop2010', 'pop10_sqmi',
       'white', 'black', 'asian', 'hispanic', 'age_under5', 'med_age',
       'households', 'families', 'ave_fam_sz', 'hse_units', 'vacant',
       'owner_occ', 'renter_occ', 'geometry', 'area'],
      dtype='object')

In [717]:
winners = results.groupby(['state_name'])\
.agg({'votes_gop': 'sum', 'votes_dem': 'sum'}).sort_values(by='state_name', ascending=True).reset_index()

In [718]:
winners

,state_name,votes_gop,votes_dem
0,Alabama,1306925,718084
1,Alaska,3781977,2697087
2,Arizona,1021154,936250
3,Arkansas,677904,378729
4,California,3916209,7362490
5,Colorado,1137455,1212209
6,Connecticut,668266,884432
7,Delaware,185103,235581
8,District of Columbia,11553,260223
9,Florida,4605515,4485745


In [745]:
results2016 = pd.read_json('https://graphics.latimes.com/la-na-pol-2016-election-results-national.json')

In [747]:
test = df['county_map'].dropna()

In [753]:
tested = pd.DataFrame(test)
tested.head()

,county_map
AK,"{'02000': {'status': {'votecount': 309407, 're..."
AL,"{'01069': {'status': {'votecount': 42315, 'rep..."
AR,"{'05109': {'status': {'votecount': 3981, 'repo..."
AZ,"{'04027': {'status': {'votecount': 52292, 'rep..."
CA,"{'06065': {'status': {'votecount': 745780, 're..."


In [767]:
from pandas.io.json import json_normalize    
df = json_normalize(tested['county_map'])